# 1 Setup

In [1]:
# System imports
import os
import re

# Environment variable management
from dotenv import load_dotenv

# HTTP requests
import requests

# Web scraping
from bs4 import BeautifulSoup

# NLP processing
import spacy

# Ranking algorithm
from rank_bm25 import BM25Okapi

# OpenAI API
from openai import OpenAI

# User interface
import gradio as gr

/Users/winsonchow/Documents/GitHub/RAG-app/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/winsonchow/Documents/GitHub/RAG-app/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the environment variables
load_dotenv()

# Retrieve the API key from the environment variable
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [3]:
# Define API endpoints
list_posts_url = 'https://www.techinasia.com/wp-json/techinasia/2.0/posts'
search_posts_url = 'https://www.techinasia.com/wp-json/techinasia/2.0/articles?query='

# 2 Data Retrieval

## 2.1 Fetch Post by Page

In [4]:
# Function to fetch posts from API
def fetch_posts_from_api(page):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(list_posts_url, params={'page': page}, headers=headers)
        response.raise_for_status()
        print("Status Code:", response.status_code)
        # print("Response Headers:", response.headers)
        # print("Response Text:", response.text)

        post_data = response.json()

        return post_data['posts']
    
    except requests.exceptions.RequestException as e:
        print(f'Error fetching posts: {e}')
        return None
    except ValueError as e:
        print(f'Error parsing response: {e}')
        return None

In [5]:
# Fetch posts
page = 1
posts = fetch_posts_from_api(page=page)
print(f"Fetched {len(posts)} posts.")

# # Display the first post to verify the structure
# if posts:
#     print(posts[0])

Status Code: 200
Fetched 30 posts.


## 2.2 Search Post by Keywords

In [6]:
# Function to search posts from API
def search_posts(keywords):
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(search_posts_url, params={'query': keywords}, headers=headers)

        if response.status_code == 200:
            try:
                return response.json()
            except ValueError as e:
                print(f'Error parsing response JSON: {e}')
                return []
        else:
            print(f'Error fetching posts: HTTP {response.status_code}')
            return []
    
    except requests.exceptions.RequestException as e:
        print(f'Error fetching posts: {e}')
        return []

# 3 Preprocessing

## 3.1 Extracting Keywords from User Query

In [7]:
# Load the spacy model
nlp = spacy.load('en_core_web_sm')

In [8]:
def preprocess_query(query):
    doc = nlp(query)
    keywords = []
    
    for token in doc:
        if not token.is_stop and not token.is_punct:
            # Add named entities and nouns to keywords list
            if token.ent_type_ or token.pos_ in ['NOUN', 'PROPN', 'ADJ']:
                keywords.append(token.lemma_)
    
    return keywords

In [9]:
# Example query
query = "What’s happening today?"
keywords = preprocess_query(query)
print("Extracted Keywords:", keywords)

query = "Tell me about Grab’s profitability."
keywords = preprocess_query(query)
print("Extracted Keywords:", keywords)

query = "Give me a history of HonestBee."
keywords = preprocess_query(query)
print("Extracted Keywords:", keywords)

query = "What are the top startups in Indonesia?"
keywords = preprocess_query(query)
print("Extracted Keywords:", keywords)

query = "What are the upcoming Events at Tech in Asia?"
keywords = preprocess_query(query)
print("Extracted Keywords:", keywords)

Extracted Keywords: ['today']
Extracted Keywords: ['Grab', 'profitability']
Extracted Keywords: ['history', 'HonestBee']
Extracted Keywords: ['startup', 'Indonesia']
Extracted Keywords: ['upcoming', 'event', 'Tech', 'Asia']


## 3.2 Extracting Relevant Articles

### 3.2.1 Retrieving All Articles

In [10]:
# Function to clean up the article content
def preprocess_article(article):
    # Remove HTML tags
    article = BeautifulSoup(article, 'html.parser').get_text()

    # Remove URLs
    article = re.sub(r'http\S+', '', article)

    # Handle special characters
    article = re.sub(r'&amp;', '&', article)

    # Normalise whitespace
    article = re.sub(r'\s+', ' ', article)

    # Remove source and citation tags
    article = re.sub(r'\[.*?\]+', '', article)

    return article

In [11]:
def fetch_and_preprocess_articles(search_results):
    # Initialize an empty list to hold the articles
    articles = []

    # Display the search results
    if search_results and 'posts' in search_results and 'hits' in search_results['posts']:
        search_posts = search_results['posts']['hits']

        # Append each article's title and content to the articles list
        articles = [post['title'] + post['content'] for post in search_posts[:30]]

        # Apply preprocessing to each article
        cleaned_articles = [preprocess_article(article) for article in articles]
    else:
        cleaned_articles = ['No relevant posts found.']

    return cleaned_articles

In [12]:
# Count the number of articles in the search results
def count_articles(search_results):
    if search_results and 'posts' in search_results and 'hits' in search_results['posts']:
        articles_count = len(search_results['posts']['hits'])
        return articles_count
    return 0

In [13]:
# Display articles
def display_articles(articles):
    for idx, article in enumerate(articles):
        print(f"Article {idx + 1}: {article}")

In [14]:
# Fetch search results
search_results = search_posts(keywords)

# Count the number of articles
articles_count = count_articles(search_results)
print(f"Number of articles found: {articles_count}")

# Fetch and preprocess articles
cleaned_articles = fetch_and_preprocess_articles(search_results)

# Display the preprocessed articles
display_articles(cleaned_articles)

Number of articles found: 30
Article 1: Southeast Asia’s up-and-coming wealthtech playersWelcome to The Top Up! Delivered every Wednesday via email and through the Tech in Asia website, this free newsletter breaks down the biggest stories and trends in fintech. Get it in your email inbox by registering here. Hello there, Since becoming a quarter-of-a-century-old, I’ve noticed that I can spend hours with my friends talking about how to manage our income. Many of us have started investing in stocks and crypto in order to achieve savings milestones before turning 30. However, I’ve also learned that financial management is a lifelong hustle. Whether you’re planning a wedding or looking to retire, budgeting is crucial. And with the help of tech-enabled wealth management platforms, we can do it more easily and efficiently. In this week’s Big Story, my colleague Qishin maps out notable wealthtech players in Southeast Asia and further categorizes them into digital brokerages, advisories, decen

### 3.2.2 Ranking Articles with BM25

In [15]:
def rank_articles(cleaned_articles, keywords):
    if cleaned_articles and cleaned_articles[0] != 'No relevant posts found.':
        # Tokenize the articles
        tokenized_articles = [article.split() for article in cleaned_articles]

        # Initialize BM25
        bm25 = BM25Okapi(tokenized_articles)

        # Tokenize the query
        tokenized_query = " ".join(keywords).split()

        # Get BM25 scores
        scores = bm25.get_scores(tokenized_query)
        
        # Get the indices of the top 5 articles
        top_5_indices = scores.argsort()[::-1][:5]
        
        # Select the top 5 articles based on BM25 scores
        top_5_articles = [cleaned_articles[i] for i in top_5_indices]

        # Display the top 5 articles
        for idx, article in enumerate(top_5_articles):
            # Display first 500 characters for brevity
            print(f"Rank {idx + 1} Article (Score: {scores[top_5_indices[idx]]}): {article[:500]}")
    else:
        print("No relevant articles found.")
        return []

In [16]:
# Rank articles
top_5_articles = rank_articles(cleaned_articles, keywords)

Rank 1 Article (Score: 7.068766846964828): Asia Partners’ co-founders break down the firm’s 2022 Internet ReportIn 2019, growth equity firm Asia Partners forecast that the market value of Southeast Asia's tech companies would soar by a mind-boggling US$425 billion within a decade, starting out with a total of US$86 billion at the time. Two years on, the region's tech firms are already halfway there. With a surge in the number of tech unicorns, a slate of upcoming public listings, a shrinking talent gap, and a growth in series C and D dea
Rank 2 Article (Score: 5.0721867447332025): Asian startups in state of tension amid coronavirus pandemic: Tech in Asia surveyAsian startups are becoming increasingly worried as the Covid-19 outbreak starts to negatively affect their businesses, according to a recent Tech in Asia survey of over 140 industry professionals and founders. About 80% of those surveyed said their businesses have been negatively influenced by the pandemic, which Sequoia Capital

## Comparing Retrieval Methods

# 4 RAG Implementation

In [ ]:
# Function to process search results and generate background knowledge
def generate_context(search_results):
    if search_results and 'posts' in search_results and 'hits' in search_results['posts']:
        search_posts = search_results['posts']['hits']

        # background knowledge with title and content
        raw_articles = "".join([post['title'] + post['content'] for post in search_posts[:3]])
    else:
        raw_articles = 'No relevant posts found.'
    return raw_articles

In [ ]:
# Function to generate responses using OpenAI
def generate_response(search_results, query):
    try:
        background_knowledge = generate_context(search_results)

        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "Answer the question based on the question asked and background knowledge provided below, and if the question can't be answered based on the context, say \"I don't know\"\n\n"},
                {"role": "user", "content": f"Question: {query}\nBackground Knowledge: {background_knowledge}\nAnswer:"}
            ],
            max_tokens=100
        )
        return response.choices[0].message.content
    
    except Exception as e:
        print(f'Error generating response: {e}')
        return None

In [ ]:
# # Test the response generation
# query = 'How is Grab performing in the market?'

# search_results = search_posts(query)
# generate_response(search_results, query)

# Interface

In [ ]:
# # Build the Gradio interface
# def rag_interface(query):
#     search_results = search_posts(query)
#     return generate_response(search_results, query)

# iface = gr.Interface(fn=rag_interface, inputs="text", outputs="text", title='Tech in Asia RAG System', description='Ask a question about Tech in Asia and get an answer based on the context of the latest posts.')
# iface.launch()